**Importing Data from google drive**

The location of the project is '/content/drive/MyDrive/concrete_ai'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/concrete_ai')
cwd = os.getcwd()
print('current directory : ', cwd)

Mounted at /content/drive
current directory :  /content/drive/MyDrive/concrete_ai


**Importing libraries**

that includes (keras, numpy, pandas and tensorflow)

also used custom classes like 
- data : for data loading
- utils : for setuping the model
- printing_callback : for good visualization of results while training

In [ ]:
import os
import data
import numpy as np
import pandas as pd
from utils import utils
import tensorflow as tf
import printing_callback as callback
from keras.layers.core import Dense
from keras.models import Sequential
from keras.layers import LayerNormalization, Dropout
from tensorflow import keras

Setting the random seed

In [ ]:
tf.keras.utils.set_random_seed(4)

Settings the input excel files path

In [ ]:
INPUT_TITLE = 'additives2'
OUTPUT_TITLE = 'additives2_28days'

**Data proccessing**

Data loading, removing old files and division into training, testing and validation 

In [ ]:
train , test, validation = data.getFinalData('data_files/' + INPUT_TITLE + '.xlsx')
train, test, validation = data.prepareMultipleData(train, test, validation, [12, 13, 14])

tmp1 = train.shape[1] - 1
tmp2 = train.shape[1]
 
xr = train[:,0:tmp1]
yr = train[:,tmp1:tmp2]
xt = test[:,0:tmp1]
yt = test[:,tmp1:tmp2]
xv = validation[:,0:tmp1]
yv = validation[:,tmp1:tmp2]
 
utils.exceptionIfNan(train)
utils.exceptionIfNan(test)
utils.exceptionIfNan(validation)
 
print('data ready with train:', train.shape, 'and test:', test.shape,
      'and validation:', validation.shape)

loading the data ...
data loaded with size:  (1075, 16) 


there is Nan in data


shuffling the data ...
data shuffled


dividing the data ...
training data ready with size: (753, 16)
test data ready with size: (162, 16)
validation data ready with size: (160, 16)
data ready with train: (350, 13) and test: (72, 13) and validation: (73, 13)


**Setting up the new modle**

In [ ]:
model = utils.newSeqentialModel(xr.shape[1], yr.shape[1])

**Loading previously saved weights**

In [ ]:
print('loading model weights ...')
output_dir = os.path.join(os.getcwd(), "saved_wights")
try:
    model.load_weights(filepath=os.path.join(output_dir, OUTPUT_TITLE + ".h5"))
    print('model weights loaded')
except OSError:
    print('no previous weights found')
except ValueError:
    print('previous weights are different from current') 

loading model weights ...
model weights loaded


**Training the model**

In [ ]:
print('Trainning model using ANN\n')
model.compile(loss='mean_squared_error', optimizer='adam', 
metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

model.fit(xr, yr, epochs=20000, batch_size=32, validation_data=(xv, yv), 
verbose=0, callbacks=[callback.LossAndErrorPrintingCallback(),
                      tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1000),])
print('\nmodel trained')

Trainning model using ANN

epoch 0 : loss :   32.89 , MPE :   12.75. validation loss :   29.64 , MPE :   12.49
epoch 100 : loss :   19.05 , MPE :    9.89. validation loss :   26.00 , MPE :   11.37
epoch 200 : loss :   20.26 , MPE :    9.97. validation loss :   25.86 , MPE :   10.81
epoch 300 : loss :   19.23 , MPE :    9.46. validation loss :   30.42 , MPE :   12.14
epoch 400 : loss :   17.80 , MPE :    9.38. validation loss :   29.69 , MPE :   11.61
epoch 500 : loss :   17.33 , MPE :    9.45. validation loss :   34.72 , MPE :   12.86
epoch 600 : loss :   15.07 , MPE :    8.53. validation loss :   34.11 , MPE :   12.85
epoch 700 : loss :   17.53 , MPE :    9.10. validation loss :   34.26 , MPE :   12.59
epoch 800 : loss :   15.64 , MPE :    8.69. validation loss :   32.71 , MPE :   12.41
epoch 900 : loss :   14.85 , MPE :    8.29. validation loss :   40.37 , MPE :   13.78
epoch 1000 : loss :   14.51 , MPE :    8.11. validation loss :   36.62 , MPE :   13.06

model trained


**Evaluating the model accuracy in the training, validation, testing sets**

In [ ]:
print('\nmodel trained')
_, accuracy_train = model.evaluate(xr, yr)
print('\nError in train data: %.2f' % (accuracy_train))

_, accuracy_validation = model.evaluate(xv, yv)
print('\nError in validation data: %.2f' % (accuracy_validation))

_, accuracy_test = model.evaluate(xt, yt)
print('\nError in test data: %.2f' % (accuracy_test))

3/3 [==============================] - 0s 6ms/step - loss: 132.4403 - mean_absolute_percentage_error: 24.6790

Error on test data: 24.68


**Testing the model in 20 random samples from the test set**

In [ ]:
test_predictions = np.around(model.predict(xt), 1)
 
for i in range(20):
    print(' test: predicted:', test_predictions[i], 'real data:', yt[i])

 test: predicted: [29.2] real data: [31.8]
 test: predicted: [30.7] real data: [26.56666667]
 test: predicted: [29.6] real data: [28.7]
 test: predicted: [27.9] real data: [36.2]
 test: predicted: [27.7] real data: [32.03333333]
 test: predicted: [27.8] real data: [38.4]
 test: predicted: [28.9] real data: [34.7]
 test: predicted: [26.9] real data: [34.]
 test: predicted: [28.9] real data: [40.8]
 test: predicted: [27.5] real data: [50.83333333]
 test: predicted: [28.] real data: [45.2]
 test: predicted: [26.9] real data: [44.9]
 test: predicted: [27.7] real data: [43.8]
 test: predicted: [27.3] real data: [38.]
 test: predicted: [28.1] real data: [43.9]
 test: predicted: [28.6] real data: [31.7]
 test: predicted: [30.2] real data: [41.6]
 test: predicted: [27.5] real data: [30.8]
 test: predicted: [27.] real data: [45.23333333]
 test: predicted: [28.1] real data: [25.06666667]


**Saving the test set output**

In [ ]:
result = np.concatenate([xt, yt, test_predictions], axis=1)
data.saveData(result, 'results/' + OUTPUT_TITLE +'.xlsx')

saved data to excel file 
path: results/additives2_7days.xlsx


**Saving the model weights**

In [ ]:
print('\nsaving model weights ...')
output_dir = os.path.join(os.getcwd(), "saved_wights")
model.save_weights(filepath=os.path.join(output_dir, OUTPUT_TITLE + ".h5"))
print('model weights saved')


saving model weights ...
model weights saved
